# OpenACC Interoperability

This lab is intended for Fortran programmers. If you prefer to use C/C++, click [this link.](../C/README.ipynb)

---
## Introduction

The primary goal of this lab is to cover how to write an OpenACC code to work alongside other CUDA codes and accelerated libraries. There are several ways to make an OpenACC/CUDA interoperable code, and we will go through them one-by-one, with a short exercise for each.

When programming in OpenACC, the distinction between CPU/GPU memory is abstracted. For the most part, you do not need to worry about explicitly differentiating between CPU and GPU pointers; the OpenACC runtime handles this for you. However, in CUDA, you do need to differentiate between these two types of pointers. Let's start with using CUDA allocated GPU data in our OpenACC code.

---

## OpenACC Deviceptr Clause

The OpenACC `deviceptr` clause is used with the `data`, `parallel`, or `kernels` directives. It can be used in the same way as other data clauses such as `copyin`, `copyout`, `copy`, or `present`. The `deviceptr` clause is used to specify that a pointer is not a host pointer but rather a device pointer.

This clause is important when working with OpenACC + CUDA interoperability because it is one way we can operate on CUDA allocated device data within an OpenACC code. Take the following example:

**Allocation with CUDA Fortran**

```fortran
integer, device, allocatable :: cuda_array(:)

allocate(cuda_array(N))
```

**Parallel Loop with OpenACC**

```fortran
!$acc parallel loop deviceptr(cuda_array)
do i = i, N
    cuda_array(i) = 0.0
end do
```

Normally, the OpenACC runtime expects to be given a host pointer, which will then be translated to some associated device pointer. However, when using CUDA to do our data management, we do not have that connection between host and device. The `deviceptr` clause is a way to tell the OpenACC runtime that a given pointer should not be translated since it is already a device pointer.

This is a situation where CUDA Fortran has an advantage over CUDA C; since in CUDA Fortran you have to specify which arrays are on the device, the OpenACC runtime is often able fill in the blanks, and produce correct code results. Regardless, it is still proper design to use the deviceptr clause in such situations, and can avoid unexpected errors and confusion down the line.

To practice using the deviceptr clause, we have a short exercise. We will examine two functions, both compute a dot product. The first code is [dot.f90](/edit/Fortran/deviceptr/dot.f90), which is a serial dot product. Next is [dot_acc.f90](/edit/Fortran/deviceptr/dot_acc.f90), which is an OpenACC parallelized version of dot. Both dot and dot_acc are called from [main.cuf](/edit/Fortran/deviceptr/main.cuf) (*note: .cuf is the conventional file extension for a CUDA Fortran source file*). In main.cu, we use host pointers to call dot, and device pointers to call dot_acc. Let's quickly run the code, it will produce an error.

In [ ]:
!make -C deviceptr

To fix this error, we must edit [dot_acc.f90](/edit/Fortran/deviceptr/dot_acc.f90) and specify that arrays `A`, `B`, and `C` are on the device (`real, device, intent(in) :: A(m,n)`, for example). And while the code may not need it to work correctly, add the deviceptr clause to the parallel loop. When you think you have it, run the code below and see if the error is fixed.

In [ ]:
!make -C deviceptr

Next, let's do the opposite. Let's take data that was allocated with OpenACC, and use it in a CUDA function.

---
## OpenACC host_data directive

The **host_data** directive is used to make the OpenACC mapped device address available to the host. There are a few clauses that can be used with host_data, but the one that we are interested in using is **use_device**. We will use the host_data directive with the use_device clause to grab the underlying device pointer that OpenACC usually abstracts for us. Then we can use this device pointer to pass to CUDA kernels or to use accelerated libraries. Let's look at a code example:

**Inside CUDA Fortran Code**

```fortran
module example_cuda_m
    attributes (global) subroutine example_kernel(A, size)
        ! Kernel Code
    end subroutine

    subroutine example_cuda(A, size)
        use cudafor
        real, device, intent(int) :: A(:)
        integer, intent(in) :: size
        
        call mydot_kernel<<<512, 128>>>(A, size)
        return
    end subroutine
end module example_cuda_m
```

**Inside OpenACC Code**

```fortran
program main
    use cuda_example_m
    
    integer, allocatable :: A(:)
    
    allocate( A(100) )
    
    !$acc data create(A(:))
        !$acc host_data use_device(A)
            call example_cuda(A, 100)
        !$acc end host_data
    !$acc end data
    
end program
```

A brief rundown of what is actually happening under-the-hood: the *data* directive creates a device copy of the array A, and the host pointer of A is linked to the device pointer of A. This is typical OpenACC behavior. Next, the *host_data use_device* translates A to the device pointer so that we can pass it to our CUDA function.

To practice this, let's work on another code. We still have [dot.f90](/edit/Fortran/host_data/dot.f90) for our serial code. But instead of an OpenACC version of dot, we have a CUDA version in [dot_cuda.cuf](/edit/Fortran/host_data/dot_cuda.cuf). Both of these functions are called in [main.f90](/edit/Fortran/host_data/main.f90). First, let's run the code and see the error.

In [ ]:
!make -C host_data

Now edit [main.90](/edit/Fortran/host_data/main.f90) and use the host_data/use_device to pass device pointers when calling our CUDA function. When you're ready, rerun the code below, and see if the error is fixed.

In [ ]:
!make -C host_data

---
## Using cuBLAS with OpenACC

We are also able to use accelerated libraries with **host_data use_device** as well. Just like the previous section, we can allocate the data with OpenACC using either the *data* or *enter data* directive. Then, pass that data to a cuBLAS call with *host_data*. This code is slightly different than before; we will be working on a matrix multiplication code. The serial code is found in [matmult.f90](/edit/Fortran/cublas/matmult.f90). The cuBLAS code is in [matmult_cublas.f90](/edit/Fortran/cublas/matmult_cublas.f90). Both of these are called from [main.f90](/edit/Fortran/cublas/main.f90). Let's try running the code and seeing the error.

In [ ]:
!make -C cublas

Now, edit [main.f90](/edit/C/cublas/main.f90) and use host_data/use_device on the cublas call (similar to what you did in the previous exercise). Rerun the code below when you're ready, and see if the error is fixed.

In [ ]:
!make -C cublas

---
## OpenACC map_data

We briefly mentioned earlier about how OpenACC creates a mapping between host and device memory. When using CUDA allocated memory within OpenACC, that mapping is not created automatically, but it can be created manually. We are able to map a host address to a device address by using the OpenACC **acc_map_data(host_array, device_array, length_in_bytes)** function. Then, before the data is unallocated, you will use **acc_unmap_data(host_array)** to undo the mapping. Let's look at a quick example.

```
real, allocatable :: A(:)
real, device, allocatable :: A_d(:)

allocate( A(N) )
allocate( A_d(N) )

call acc_map_data(A, A_d, N*4)

!$acc parallel loop present(A(:))
do i = 1, N
    A(i) = 0.0
end do

call acc_unmap_data(A)

deallocate( A )
deallocate( A_d )
```

To practice, we have another example code which uses the dot functions again. Serial dot is in [dot.f90](/edit/Fortran/map/dot.f90). OpenACC dot is in [dot_acc.f90](/edit/Fortran/map/dot_acc.f90). Both of them are called from [main.cuf](/edit/Fortran/map/main.cuf). Try running the code and see the error.

In [ ]:
!make -C map

Now, edit [main.cuf](/edit/Fortran/map/main.cuf) and add the OpenACC mapping functions before any of our OpenACC directives. When you're ready, rerun the code below and see if the error is fixed.

In [ ]:
!make -C map

---
## Routine

The last topic to discuss is using CUDA `__device__` functions within OpenACC parallel and kernels regions. These are functions that are compiled to be called from the accelerator within a GPU kernel or OpenACC region.

If you want to compile an OpenACC function to be used on the device, you will use the **routine** directive with the following syntax:

```fortran
real function sqab(a)
    !$acc routine seq
    real :: a 
    sqab = sqrt(abs(a))
end function
```

You can also have a function with a loop you want to parallelize like so:

```fortran
subroutine test( x, n )
    !$acc routine vector
    real, dimension(*) :: x
    integer :: n
    integer :: i
    
    !$acc loop vector
    do i=1, n
        x(i) = 0.0
    enddo
```

To use CUDA device functions within our OpenACC loops, we can also use the routine directive. See the following example:

**In CUDA Code**

```fortran
module dist_cuda_m
contains
    attributes(device) function dist_cuda(a, b)
        !$acc routine seq
        real :: a, b

        dist_cuda = sqrt(a*a + b*b)
    end function dist_cuda
end module dist_cuda_m
```
**In OpenACC Code**

```fortran
module distance_map_acc_m
    use  dist_cuda_m
    use cudafor
    
contains
    subroutine distance_map_acc(A, B, C, m, n)
        implicit none
        real, intent(in) :: A(:)
        real, intent(in) :: B(:)
        real, intent(inout) :: C(:,:)
        integer, intent(in) :: m, n
         
        integer :: i, j

        !$acc parallel loop copyin(A, B) copyout(C)
        do j = 1,m
            !$acc loop
            do i = 1,n
                C(i,j) = dist_cuda(A(j), B(i) )
            end do
        end do
    end subroutine distance_map_acc
end module distance_map_acc_m
```

Fortunately, if you're using CUDA Fortran (and depending on the compiler) you may not need to even include explicit routine information. However, it is considered proper to include routine information in these kinds of situations.

---
## Bonus Task

Here are some additional resources for OpenACC/CUDA interoperability:

[This is an NVIDIA devblog about some common techniques for implementing OpenACC + CUDA](https://devblogs.nvidia.com/3-versatile-openacc-interoperability-techniques/)

[This is a github repo with some additional code examples demonstrating the lessons covered in this lab](https://github.com/jefflarkin/openacc-interoperability)

---

## Post-Lab Summary

If you would like to download this lab for later viewing, it is recommend you go to your browsers File menu (not the Jupyter notebook file menu) and save the complete web page.  This will ensure the images are copied down as well.

You can also execute the following cell block to create a zip-file of the files you've been working on, and download it with the link below.

In [ ]:
%%bash
rm -f openacc_files.zip
zip -r openacc_files.zip *

**After** executing the above zip command, you should be able to download the zip file [here](files/openacc_files.zip)